In [1]:
%%writefile inference.py
import joblib
import os
import json
from scipy.sparse import hstack, csr_matrix
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pickle
import boto3


"""
Deserialize fitted model
"""
def model_fn(model_dir):
    # Load the trained model from the model directory
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    return model

"""
input_fn
    request_body: The body of the request sent to the model.
    request_content_type: (string) specifies the format/variable type of the request
"""
def input_fn(request_body, request_content_type):
    if request_content_type == 'application/json':
        # Parse the input data if it's in JSON format
        input_data = json.loads(request_body)
        return input_data
    else:
        raise ValueError("This model only supports application/json input")

"""
predict_fn
    input_data: returned data from input_fn above
    model (sklearn model) returned model loaded from model_fn above
"""
def predict_fn(input_data, model):
    # Process the input data if necessary
    processed_data = process_input(input_data)
    # Make predictions using the model
    predictions = model.predict(processed_data)
    return predictions

def process_input(input_data):
    # Extract necessary data from the input dictionary
    NgramFeaturesList_pred = np.array(input_data['NgramFeaturesList_pred'])
    importsCorpus_pred = input_data['importsCorpus_pred']
    sectionNames_pred = input_data['sectionNames_pred']
    numSections_pred = int(input_data['numSections_pred'])

    # Load featurizers
    imports_featurizer = joblib.load(os.path.join("opt/ml/model", "imports_featurizer.pkl"))
    section_names_featurizer = joblib.load(os.path.join("opt/ml/model", "section_names_featurizer.pkl"))
    
    # Transform text features
    importsCorpus_pred_transformed = imports_featurizer.transform([importsCorpus_pred])
    sectionNames_pred_transformed = section_names_featurizer.transform([sectionNames_pred])

    # Concatenate features into a single sparse matrix
    processed_data = hstack([csr_matrix(NgramFeaturesList_pred),
                             importsCorpus_pred_transformed,
                             sectionNames_pred_transformed,
                             csr_matrix([numSections_pred]).transpose()])
    return processed_data

"""
output_fn
    prediction: the returned value from predict_fn above
    content_type: the content type the endpoint expects to be returned. Ex: JSON, string
"""
def output_fn(prediction, content_type):
    # Prepare the output in JSON format
    res = int(prediction[0])
    respJSON = {'Output': res}
    return respJSON

"""if __name__ == '__main__':
    # Example usage for testing the functions individually
    predict_fn({'NgramFeaturesList_pred': [[24183, 3382, 304, 17, 923, 636, 358, 275, 128, 635, 358, 613, 389, 384, 448, 12, 380, 170, 307, 122, 224, 203, 51, 338, 521, 111, 395, 215, 175, 419, 264, 397, 287, 106, 487, 236, 16, 277, 459, 594, 469, 241, 155, 163, 158, 230, 215, 443, 80, 46, 44, 216, 68, 42, 36, 48, 161, 29, 240, 145, 139, 52, 20, 75, 99, 33, 224, 161, 38, 226, 729, 139, 27, 168, 19, 68, 269, 271, 236, 33, 197, 207, 337, 1114, 126, 111, 255, 175, 47, 46, 60, 318, 129, 79, 16, 223, 162, 79, 15, 157]],
 'importsCorpus_pred': "kernel32 shlwapi ole32 shell32 user32",
 'sectionNames_pred': ".text .rdata .data .rsrc .reloc",
 'numSections_pred': "5"}, model_fn(""))
"""

Overwriting inference.py


In [2]:
%%writefile requirements.txt
joblib
scipy
numpy
scikit-learn

Overwriting requirements.txt


In [15]:
!mkdir -p opt/ml/model  
!cp model.joblib opt/ml/model/model.joblib
!cp imports_featurizer.pkl opt/ml/model/imports_featurizer.pkl
!cp section_names_featurizer.pkl opt/ml/model/section_names_featurizer.pkl

In [4]:
!tar -cvpzf model.tar.gz model.joblib inference.py imports_featurizer.pkl section_names_featurizer.pkl requirements.txt opt

model.joblib
inference.py
imports_featurizer.pkl
section_names_featurizer.pkl
requirements.txt
opt/
opt/ml/
opt/ml/model/
opt/ml/model/imports_featurizer.pkl
opt/ml/model/model.pkl
opt/ml/model/model.joblib
opt/ml/model/section_names_featurizer.pkl


In [5]:
import os
import json
import boto3
import joblib
import pickle
import tarfile
import sagemaker
from sagemaker.estimator import Estimator
import time
from time import gmtime, strftime
import subprocess

In [6]:
# Setting up Boto3 clients and resources
client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")
boto_session = boto3.session.Session()
s3 = boto_session.resource('s3')

# Retrieving the AWS region
region = boto_session.region_name
print(region)

# Creating a SageMaker session
sagemaker_session = sagemaker.Session()

# Defining the IAM role for SageMaker
role = "arn:aws:iam::533267294611:role/LabRole"

us-east-1


In [7]:
# Retrieve the URI for the Scikit-learn container image
image_uri = sagemaker.image_uris.retrieve(
    framework="sklearn",          # Specify the framework as Scikit-learn
    region=region,                # Specify the AWS region
    version="1.2-1",              # Specify the version of Scikit-learn
    py_version="py3",             # Specify the Python version
    instance_type="ml.t2.medium", # Specify the instance type for hosting the model
)

In [8]:
# Get the default bucket for storing model artifacts
default_bucket = sagemaker_session.default_bucket()
print(default_bucket)

# Specify the path for uploading model artifacts to the bucket
model_artifacts = f"s3://{default_bucket}/model.tar.gz"

# Upload the model artifacts (e.g., 'model.tar.gz') to the specified bucket
response = s3.meta.client.upload_file('model.tar.gz', default_bucket, 'model.tar.gz')

sagemaker-us-east-1-533267294611


In [9]:
# Step 1: Model Creation

# Generate a unique model name with a timestamp
model_name = "sklearn-test-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# Create a model using the SageMaker client
create_model_response = client.create_model(
    ModelName=model_name,                            # Specify the name for the model
    Containers=[                                     # Specify container configuration
        {
            "Image": image_uri,                      # Specify the container image URI
            "Mode": "SingleModel",                   # Specify the mode as SingleModel
            "ModelDataUrl": model_artifacts,         # Specify the location of model artifacts
            "Environment": {                         # Specify environment variables
                'SAGEMAKER_SUBMIT_DIRECTORY': model_artifacts,
                'SAGEMAKER_PROGRAM': 'inference.py'
            } 
        }
    ],
    ExecutionRoleArn=role,                           # Specify the execution role ARN
)

# Print the model ARN
print("Model Arn: " + create_model_response["ModelArn"])

Model name: sklearn-test2024-04-03-02-48-21
Model Arn: arn:aws:sagemaker:us-east-1:533267294611:model/sklearn-test2024-04-03-02-48-21


In [10]:
# Step 2: EPC Creation

# Generate a unique endpoint configuration name with a timestamp
sklearn_epc_name = "sklearn-epc-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# Create an endpoint configuration using the SageMaker client
endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=sklearn_epc_name,    # Specify the name for the endpoint configuration
    ProductionVariants=[                    # Specify production variants
        {
            "VariantName": "sklearnvariant",            # Specify variant name
            "ModelName": model_name,                     # Specify the model name
            "InstanceType": "ml.t2.medium",              # Specify the instance type
            "InitialInstanceCount": 1                     # Specify the initial instance count
        },
    ],
)

# Print the endpoint configuration ARN
print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

Endpoint Configuration Arn: arn:aws:sagemaker:us-east-1:533267294611:endpoint-config/sklearn-epc2024-04-03-02-48-25


In [11]:
# Step 3: EP Creation

# Generate a unique endpoint name with a timestamp
endpoint_name = "sklearn-local-ep-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

# Create an endpoint using the SageMaker client
create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,            # Specify the name for the endpoint
    EndpointConfigName=sklearn_epc_name,  # Specify the endpoint configuration name
)

# Print the endpoint ARN
print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

Endpoint Arn: arn:aws:sagemaker:us-east-1:533267294611:endpoint/sklearn-local-ep2024-04-03-02-48-26


In [12]:
# Monitor creation

# Describe the endpoint to monitor its status
describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

# Check if the endpoint status is "Creating"
while describe_endpoint_response["EndpointStatus"] == "Creating":
    # Wait for 15 seconds
    time.sleep(15)
    # Describe the endpoint again to check its status
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    # Print the current status of the endpoint
    print(describe_endpoint_response["EndpointStatus"])

# Once the endpoint status is not "Creating", print the final response
print(describe_endpoint_response)


Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService
{'EndpointName': 'sklearn-local-ep2024-04-03-02-48-26', 'EndpointArn': 'arn:aws:sagemaker:us-east-1:533267294611:endpoint/sklearn-local-ep2024-04-03-02-48-26', 'EndpointConfigName': 'sklearn-epc2024-04-03-02-48-25', 'ProductionVariants': [{'VariantName': 'sklearnvariant', 'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3', 'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:0381fc23e5c6ee7a0a944240dbaf6041688cb1e439af23e0c930da072ac9e971', 'ResolutionTime': datetime.datetime(2024, 4, 3, 2, 48, 28, 10000, tzinfo=tzlocal())}], 'CurrentWeight': 1.0, 'DesiredWeight': 1.0, 'CurrentInstanceCount': 1, 'DesiredInstanceCount': 1}], 'EndpointStatus': 'InService', 'CreationTime': datetime.datetime(20